In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings 
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *
from statsforecast import StatsForecast
from statsforecast.models import (
    Naive,WindowAverage, ARIMA, 
    AutoARIMA,SeasonalNaive,HoltWinters,
    CrostonClassic as Croston, HistoricAverage,DynamicOptimizedTheta as DOT,
    SeasonalNaive
)
from dataclasses import dataclass
from typing import Optional, List
from itertools import product

warnings.filterwarnings("ignore")  # To ignore warnings from pandas/numpy

In [ ]:
user="lholguin"

In [ ]:
#New data class created to handle configuration parameters
@dataclass
class ForecastConfig:
    
    # Forecast parameters
    h: int = 8                          
    season_length: int = 4              
    
    # Cross-validation parameters
    n_windows: int = 2                  
    step_size: Optional[int] = None     
    
    # Train-test split parameters
    train_size: Optional[int] = None    # Use all available data except test
    test_size: Optional[int] = None     # Auto-set to h in __post_init__
    
    # Plotting parameters
    n_samples: int = 4                  # Plot 4 random samples
    models_to_plot: Optional[List[str]] = None  # 
    
    # Other settings
    confidence_level: int = 95          # 95% confidence intervals
    n_jobs: int = -1                    
    
    def __post_init__(self):
        
        if self.step_size is None:
            self.step_size = self.h
        if self.test_size is None:
            self.test_size = self.h
        if self.models_to_plot is None:
            self.models_to_plot = ['Naive', 'ARIMA_manual', 'SARIMA']

In [ ]:
def get_models(config):
    
    models = [Naive(), HistoricAverage(), WindowAverage(window_size=4),
        SeasonalNaive(season_length=4), ARIMA(order=(1, 1, 1), alias="ARIMA_manual"),
        AutoARIMA(seasonal=True, season_length=4, alias="SARIMA"),
    ]
    return models

def evaluate_train_test(df, target_name, config):

    if config.train_size is None:
        train = df.groupby('unique_id').apply(
            lambda x: x.iloc[:-config.test_size]
        ).reset_index(drop=True)
    else:
        train = df.groupby('unique_id').apply(
            lambda x: x.iloc[-(config.train_size + config.test_size):-config.test_size]
        ).reset_index(drop=True)
    
    test = df.groupby('unique_id').apply(
        lambda x: x.iloc[-config.test_size:]
    ).reset_index(drop=True)
    
    sf = StatsForecast(
        models=get_models(config),
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )
    sf.fit(df=train)
    preds = sf.predict(h=config.h)

    preds_df = pd.merge(test, preds.reset_index(), on=['ds', 'unique_id'], how='left')
    models = [col for col in preds.columns if col not in ['unique_id', 'ds']]
    
    eval_df = evaluate(preds_df, metrics=[mae, mse, rmse], models=models)
    
    mae_df = eval_df[eval_df['metric'] == 'mae'].copy()
    mae_df['best_model'] = mae_df[models].idxmin(axis=1)
    
    print(f"\n📈 Best Models (Train-Test Split - based on MAE):")
    print(mae_df['best_model'].value_counts())
    
    return eval_df, preds_df, train, test 

def evaluate_model_cross(df, target_name, config):
    
    sf = StatsForecast(
        models=get_models(config),
        freq='QS',
        n_jobs=config.n_jobs,
        fallback_model=SeasonalNaive(season_length=config.season_length)
    )

    print(f"   Running cross-validation...")
    cv_df = sf.cross_validation(
        df=df,
        h=config.h,
        n_windows=config.n_windows,
        step_size=config.step_size
    )
    
    # Define model columns
    exclude_cols = ['unique_id', 'ds', 'y', 'cutoff', 'metric']
    model_cols = [col for col in cv_df.columns if col not in exclude_cols]

    # Evaluate per cutoff window
    all_results = []
    
    for cutoff in cv_df['cutoff'].unique():
        cutoff_data = cv_df[cv_df['cutoff'] == cutoff]
        
        # Evaluate all metrics for this cutoff
        cutoff_eval = evaluate(cutoff_data, metrics=[mae, mse, rmse], models=model_cols)
        cutoff_eval['cutoff'] = cutoff
        
        # Add best_model column (lowest value per row, regardless of metric)
        cutoff_eval['best_model'] = cutoff_eval[model_cols].idxmin(axis=1)
        cutoff_eval['best_value'] = cutoff_eval[model_cols].min(axis=1)
        
        all_results.append(cutoff_eval)
        
        # Print MAE summary for this cutoff
        cutoff_mae = cutoff_eval[cutoff_eval['metric'] == 'mae']
        print(f"\n   Cutoff {cutoff.strftime('%Y-%m-%d')} (MAE best models):")
        print(f"   {cutoff_mae['best_model'].value_counts().to_dict()}")

    # Combine all cutoff results into single dataframe
    eval_df = pd.concat(all_results, ignore_index=True)
    
    # Print overall summary
    mae_overall = eval_df[eval_df['metric'] == 'mae']
    print(f"\n📈 Overall Best Models (all cutoffs - based on MAE):")
    print(mae_overall['best_model'].value_counts())
    
    return eval_df, cv_df

def sensitivity_analysis_with_production_forecasts(df, target_name, param_grid=None, production_horizons=[4, 8], save_path=None):
    
    # Default parameter grid if not provided
    if param_grid is None:
        param_grid = {
            'h': [4, 8],
            'test_size': [4, 8],
            'train_size': [None, 20, 28],
            'n_windows': [2, 3, 4],
            'step_size': [None, 2, 4],
        }
   
    print("\nPHASE 1: SENSITIVITY ANALYSIS")
    
    param_names = list(param_grid.keys())
    param_values = list(param_grid.values())
    combinations = list(product(*param_values))
    
    print(f"Testing {len(combinations)} configurations...")
    
    all_results = []
    
    for i, combo in enumerate(combinations):
        params = dict(zip(param_names, combo))
        
        # Skip invalid combinations
        if params.get('test_size') and params.get('h'):
            if params['test_size'] < params['h']:
                continue
        
        print(f"\nConfiguration {i+1}/{len(combinations)}: {params}")
        
        try:
            config = ForecastConfig(
                h=params.get('h', 8),
                train_size=params.get('train_size'),
                test_size=params.get('test_size'),
                n_windows=params.get('n_windows', 2),
                step_size=params.get('step_size'),
                season_length=4,
                n_samples=4,
                confidence_level=95,
                n_jobs=-1
            )
            
            # Run train-test evaluation
            eval_traintest, preds_traintest, train_df, test_df = evaluate_train_test(
                df, target_name, config
            )
            
            # Run cross-validation
            eval_cv, cv_df = evaluate_model_cross(df, target_name, config)
            
            # Extract MAE results
            mae_cv = eval_cv[eval_cv['metric'] == 'mae'].copy()
            mae_traintest = eval_traintest[eval_traintest['metric'] == 'mae'].copy()
            
            # Get model columns
            exclude_cols = ['unique_id', 'ds', 'y', 'cutoff', 'metric', 'best_model', 'best_value']
            model_cols = [col for col in mae_cv.columns if col not in exclude_cols]
            
            # Add best_model to train-test if not present
            if 'best_model' not in mae_traintest.columns:
                mae_traintest['best_model'] = mae_traintest[model_cols].idxmin(axis=1)
            
            # Store results
            for uid in df['unique_id'].unique():
                uid_cv = mae_cv[mae_cv['unique_id'] == uid]
                uid_traintest = mae_traintest[mae_traintest['unique_id'] == uid]
                
                cv_best_model = uid_cv['best_model'].mode().iloc[0] if len(uid_cv) > 0 else None
                cv_best_count = (uid_cv['best_model'] == cv_best_model).sum()
                cv_total = len(uid_cv)
                cv_consistency = cv_best_count / cv_total if cv_total > 0 else 0
                
                traintest_best_model = uid_traintest['best_model'].iloc[0] if len(uid_traintest) > 0 else None
                
                result_row = {
                    'unique_id': uid,
                    'config_id': i + 1,
                    **params,
                    'cv_best_model': cv_best_model,
                    'cv_consistency': cv_consistency,
                    'traintest_best_model': traintest_best_model,
                    'cv_traintest_agree': cv_best_model == traintest_best_model,
                }
                
                # Add MAE values for each model
                for model in model_cols:
                    model_mae = uid_cv[model].mean() if len(uid_cv) > 0 else None
                    result_row[f'{model}_mae'] = model_mae
                
                all_results.append(result_row)
            
            print(f"   ✓ Completed successfully")
            
        except Exception as e:
            print(f"   ✗ Error: {str(e)}")
            continue
    
    results_df = pd.DataFrame(all_results)
    
    print("\nPHASE 2: IDENTIFYING BEST MODELS PER HORIZON")
    recommendations_per_horizon = {}
    
    for horizon in production_horizons:
        print(f"\n🎯 Analyzing h={horizon}...")
        
        # Filter results for this horizon
        horizon_results = results_df[results_df['h'] == horizon].copy()
        
        if len(horizon_results) == 0:
            print(f"   ⚠️ No results found for h={horizon}, skipping")
            continue
        
        # Get recommendations for each unique_id at this horizon
        horizon_recommendations = []
        
        for uid in horizon_results['unique_id'].unique():
            uid_data = horizon_results[horizon_results['unique_id'] == uid]
            
            # Most frequent CV best model
            cv_mode = uid_data['cv_best_model'].mode()
            cv_best = cv_mode.iloc[0] if len(cv_mode) > 0 else None
            cv_freq = (uid_data['cv_best_model'] == cv_best).sum() / len(uid_data)
            
            # Most frequent train-test best model
            tt_mode = uid_data['traintest_best_model'].mode()
            tt_best = tt_mode.iloc[0] if len(tt_mode) > 0 else None
            tt_freq = (uid_data['traintest_best_model'] == tt_best).sum() / len(uid_data)
            
            # Average consistency
            avg_consistency = uid_data['cv_consistency'].mean()
            
            # Determine recommendation
            if cv_best == tt_best and cv_freq >= 0.7 and avg_consistency >= 0.7:
                confidence = 'High'
                recommended_model = cv_best
                reason = "CV and Train-Test agree, high frequency and consistency"
            elif cv_best == tt_best and cv_freq >= 0.5:
                confidence = 'Medium-High'
                recommended_model = cv_best
                reason = "CV and Train-Test agree with moderate frequency"
            elif cv_freq >= 0.6:
                confidence = 'Medium'
                recommended_model = cv_best
                reason = f"CV favors {cv_best} ({cv_freq:.0%})"
            elif tt_freq >= 0.6:
                confidence = 'Medium'
                recommended_model = tt_best
                reason = f"Train-Test favors {tt_best} ({tt_freq:.0%})"
            else:
                confidence = 'Low'
                mae_cols = [col for col in uid_data.columns if col.endswith('_mae')]
                if mae_cols:
                    avg_maes = uid_data[mae_cols].mean()
                    recommended_model = avg_maes.idxmin().replace('_mae', '')
                else:
                    recommended_model = cv_best
                reason = "No clear winner - using lowest average MAE"
            
            # Get average MAE
            rec_mae_col = f'{recommended_model}_mae'
            avg_mae = uid_data[rec_mae_col].mean() if rec_mae_col in uid_data.columns else None
            
            recommendation = {
                'unique_id': uid,
                'horizon': horizon,
                'recommended_model': recommended_model,
                'confidence': confidence,
                'reason': reason,
                'cv_best_model': cv_best,
                'cv_frequency': cv_freq,
                'cv_consistency': avg_consistency,
                'traintest_best_model': tt_best,
                'avg_mae': avg_mae
            }
            
            horizon_recommendations.append(recommendation)
            
            print(f"   {uid}: {recommended_model} (confidence: {confidence}, MAE: {avg_mae:,.0f})")
        
        recommendations_per_horizon[horizon] = pd.DataFrame(horizon_recommendations)
    

    print("\nPHASE 3: GENERATING PRODUCTION FORECASTS")
    
    production_forecasts = {}
    
    for horizon in production_horizons:
        if horizon not in recommendations_per_horizon:
            continue
            
        print(f"\n🔮 Generating forecasts for h={horizon}...")
        
        recommendations = recommendations_per_horizon[horizon]
        
        # Train on FULL dataset
        config_prod = ForecastConfig(
            h=horizon,
            season_length=4,
            confidence_level=95,
            n_jobs=-1
        )
        
        # Train all models on full data
        sf = StatsForecast(
            models=get_models(config_prod),
            freq='QS',
            n_jobs=-1,
            fallback_model=SeasonalNaive(season_length=4)
        )
        
        forecasts_df = sf.forecast(df=df, h=horizon, level=[95])
        
        # Create a "best model" forecast by selecting the recommended model for each unique_id
        best_forecasts = []
        
        for uid in df['unique_id'].unique():
            uid_forecasts = forecasts_df.reset_index()
            uid_forecasts = uid_forecasts[uid_forecasts['unique_id'] == uid]
            
            uid_rec = recommendations[recommendations['unique_id'] == uid]
            if len(uid_rec) == 0:
                print(f"   ⚠️ No recommendation found for {uid}, skipping")
                continue
            
            best_model = uid_rec['recommended_model'].iloc[0]
            
            for _, row in uid_forecasts.iterrows():
                best_row = {
                    'unique_id': uid,
                    'ds': row['ds'],
                    'recommended_model': best_model,
                    'forecast': row.get(best_model, np.nan),
                    'forecast_lo_95': row.get(f'{best_model}-lo-95', np.nan),
                    'forecast_hi_95': row.get(f'{best_model}-hi-95', np.nan)
                }
                best_forecasts.append(best_row)
        
        best_forecasts_df = pd.DataFrame(best_forecasts)
        
        production_forecasts[horizon] = {
            'all_models': forecasts_df,
            'best_model': best_forecasts_df,
            'recommendations': recommendations
        }
        
        print(f"   ✓ Generated {len(best_forecasts_df)} forecast periods")
        print(f"   ✓ Models used: {recommendations['recommended_model'].value_counts().to_dict()}")
    
        print("\nPHASE 4: SAVING RESULTS")
        
        os.makedirs(save_path, exist_ok=True)
        
        # Save sensitivity analysis results
        sensitivity_file = os.path.join(save_path, f'{target_name.lower().replace(" ", "_")}_sensitivity_analysis.xlsx')
        with pd.ExcelWriter(sensitivity_file, engine='openpyxl') as writer:
            results_df.to_excel(writer, sheet_name='All_Configurations', index=False)
            
            # Combined recommendations
            all_recommendations = pd.concat(recommendations_per_horizon.values(), ignore_index=True)
            all_recommendations.to_excel(writer, sheet_name='Recommendations', index=False)
        
        print(f"✓ Sensitivity analysis saved: {sensitivity_file}")
        
        # Save production forecasts for each horizon
        for horizon, forecasts in production_forecasts.items():
            horizon_file = os.path.join(save_path, f'{target_name.lower().replace(" ", "_")}_production_h{horizon}.xlsx')
            
            with pd.ExcelWriter(horizon_file, engine='openpyxl') as writer:
                # All models
                forecasts['all_models'].reset_index().to_excel(
                    writer, sheet_name='All_Models', index=False
                )
                
                # Best model forecasts
                forecasts['best_model'].to_excel(
                    writer, sheet_name='Best_Model_Forecast', index=False
                )
                
                # Recommendations
                forecasts['recommendations'].to_excel(
                    writer, sheet_name='Model_Selection', index=False
                )
            
            print(f"✓ Production forecasts (h={horizon}) saved: {horizon_file}")
    
    print("\n✅ COMPLETE!")
    
    return {
        'sensitivity_results': results_df,
        'production_forecasts': production_forecasts,
        'recommendations_per_horizon': recommendations_per_horizon
    }

### Population Forecast

In [ ]:
def load_data(filepath, states=None):
    
    df=pd.read_csv(filepath)
    #Filtering for certain years
    df = df[(df['Period'] >= '2017Q1') & (df['Period'] <= '2024Q4')].copy()
    if states is not None:
        if isinstance(states, str):
            states = [states]
        print(f"Filtering data for states: {states}")
        df = df[df['State'].isin(states)].copy()
        print(f"Filtered to {len(df)} rows across {df['State'].unique()} state(s)")
        if len(df) == 0:
            raise ValueError("No data available after filtering by states.")    
    
    df['unique_id']=df['State']
    df['ds']=pd.to_datetime(df['Period'])
    df=df.sort_values(['unique_id','ds']).reset_index(drop=True)
    df_pop=df[['unique_id','ds','Population']].copy()
    df_pop.columns = ['unique_id', 'ds', 'y']
    

    return df_pop,df

In [ ]:
pop_csv_path=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\Prebuilt_panels\medi_pop.csv"
pop_save_path=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\Forecast\Pop\\"

df_pop, df_original = load_data(filepath=pop_csv_path, states=['IN'])
param_grid = {
    'h': [4, 8],
    'test_size': [4, 8],
    'train_size': [None, 26],
    'n_windows': [2, 3],
    'step_size': [4],
}
results = sensitivity_analysis_with_production_forecasts(
    df=df_pop,
    target_name="Population",
    param_grid=param_grid,
    production_horizons=[4, 8],
    save_path=pop_save_path
)

### Statistical Models - Dataset with pop

In [ ]:
def load_and_prepare_data(filepath, states=None):

    df = pd.read_csv(filepath)
    #Cutting the dataframe fo selected years
    df=df[(df['Period'] >= '2017Q1') & (df['Period'] <= '2024Q4')].copy()
    
    if states is not None:
        if isinstance(states, str):
            states = [states]
        print(f"Filtering data for states: {states}")
        df=df[df['State'].isin(states)].copy()
        print(f"Filtered to {len(df)} rows across {df['State'].nunique()} state(s).")

        if len(df) == 0:
            raise ValueError("No data available for the specified states.")
    
    df['unique_id'] = df['State'] + '_' + df['ATC2 Class']
    df['ds'] = pd.to_datetime(df['Period'])
    df = df.sort_values(['unique_id','ds']).reset_index(drop=True)

    #Preparing dataframes for statsforecast
    df_units=df[['unique_id','ds','Units Reimbursed']].copy()
    df_units.columns=['unique_id','ds','y']

    df_prescriptions=df[['unique_id','ds','Number of Prescriptions']].copy()
    df_prescriptions.columns=['unique_id','ds','y']

    return df_units, df_prescriptions, df

In [ ]:
def main(filepath, states=None, config=None, run_cv=True, save_path=None):
    
    # Create default config if not provided
    if config is None:
        config = ForecastConfig()
    
    print("\n1. Loading and preparing data...")
    df_units, df_prescriptions, df_original = load_and_prepare_data(filepath, states=states)
    
    # Display summary
    print(f"\nData Summary:")
    print(f"  • Total unique series: {df_units['unique_id'].nunique()}")
    min_date = df_units['ds'].min()
    max_date = df_units['ds'].max()
    print(f"  • Date range: {min_date.year}-Q{min_date.quarter} to {max_date.year}-Q{max_date.quarter}")
    if states is not None:
        print(f"  • Filtered states: {', '.join(states)}")
    

    print("\n" + "="*70)
    print("UNITS REIMBURSED")
    print("="*70)
    
    print("\n2. Training models and generating forecasts...")
    forecasts_units, sf_units = train_and_forecast(df_units, "Units Reimbursed", config)

    print("\n3. Evaluating models on training/test split...")
    eval_traintest_units, preds_traintest_units, train_units, test_units = evaluate_train_test(
        df_units, "Units Reimbursed", config
    )
    
    if run_cv:
        print("\n4. Running cross-validation...")
        eval_cv_units, cv_units, eval_summary_units, per_cutoff_units = evaluate_model_cross(
            df_units, "Units Reimbursed", config
        )
        
        print("\n5. Selecting best forecasts...")
        best_forecasts_units = get_best_model_forecast(forecasts_units, eval_summary_units)
        
        print("\n6. Saving results...")
        save_results_to_excel(
            forecasts_units, best_forecasts_units, 
            eval_cv_units, eval_traintest_units, 
            "Units Reimbursed", save_path=save_path
        )
    else:
        eval_cv_units = None
        per_cutoff_units = None
        best_forecasts_units = forecasts_units.reset_index()
        print("\n Skipping cross-validation (run_cv=False)")
    
    # Plot sample forecasts
    print("\nGenerating forecast plots...")
    plot_sample_forecasts(df_units, forecasts_units, "Units Reimbursed", config)
    plot_train_test_forecasts(df_units, train_units, test_units, preds_traintest_units, "Units Reimbursed", config)
    

    print("\n" + "="*70)
    print("NUMBER OF PRESCRIPTIONS")
    print("="*70)
    
    print("\n2. Training models and generating forecasts...")
    forecasts_prescriptions, sf_prescriptions = train_and_forecast(
        df_prescriptions, "Number of Prescriptions", config
    )

    print("\n3. Evaluating models on training/test split...")
    eval_traintest_prescriptions, preds_traintest_prescriptions, train_prescriptions, test_prescriptions = evaluate_train_test(
        df_prescriptions, "Number of Prescriptions", config
    )
    
    if run_cv:
        print("\n4. Running cross-validation...")
        eval_cv_prescriptions, cv_prescriptions, eval_summary_prescriptions, per_cutoff_prescriptions = evaluate_model_cross(
            df_prescriptions, "Number of Prescriptions", config
        )
        
        print("\n5. Selecting best forecasts...")
        best_forecasts_prescriptions = get_best_model_forecast(
            forecasts_prescriptions, eval_summary_prescriptions
        )
        
        print("\n6. Saving results...")
        save_results_to_excel(
            forecasts_prescriptions, best_forecasts_prescriptions, 
            eval_cv_prescriptions, eval_traintest_prescriptions, 
            "Number of Prescriptions", save_path=save_path
        )
    else:
        eval_cv_prescriptions = None
        per_cutoff_prescriptions = None
        best_forecasts_prescriptions = forecasts_prescriptions.reset_index()
        print("\n Skipping cross-validation (run_cv=False)")
    
    # Plot sample forecasts
    print("\nGenerating forecast plots...")
    plot_sample_forecasts(df_prescriptions, forecasts_prescriptions, "Number of Prescriptions", config)
    plot_train_test_forecasts(df_prescriptions, train_prescriptions, test_prescriptions, preds_traintest_prescriptions, "Number of Prescriptions", config)
    
    print("\n" + "="*70)
    print("✅ PIPELINE COMPLETE!")
    print("="*70)
    
    state_info = f"{', '.join(states)}" if states else "ALL"
    print(f"\nForecasts generated for state(s): {state_info}")
    print(f"Forecast horizon: {config.h} quarters ahead")
    
    if run_cv:
        print(f"\nAll files generated!")
    
    return {
        'units': {
            'forecasts': forecasts_units,
            'best_forecasts': best_forecasts_units if run_cv else None,
            'evaluation_cv': eval_cv_units if run_cv else None,
            'evaluation_traintest': eval_traintest_units,
            'predictions_traintest': preds_traintest_units,
            'per_cutoff_best': per_cutoff_units if run_cv else None,
        },
        'prescriptions': {
            'forecasts': forecasts_prescriptions,
            'best_forecasts': best_forecasts_prescriptions if run_cv else None,
            'evaluation_cv': eval_cv_prescriptions if run_cv else None,
            'evaluation_traintest': eval_traintest_prescriptions,
            'predictions_traintest': preds_traintest_prescriptions,
            'per_cutoff_best': per_cutoff_prescriptions if run_cv else None,
        }
    }

In [ ]:
path_save=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\Forecast\TMF\\"
os.path.isdir(path_save)

In [ ]:
#path to the pre-built dataset
csv_path=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\merged_data\Prebuilt_panels\P1_nopop.csv"
path_save=rf"C:\Users\{user}\OneDrive - purdue.edu\VS code\Data\ATC\Forecast\TMF\\"

if __name__ == "__main__":
    main(filepath=csv_path, states=['IN'], h=8, run_cv=True, n_windows=5,save_path=path_save)
  